# Working with *image_dataset_from_directory()*.

The first approach in this TFM was to serialize the data in a pickle and load the whole in memory => https://github.com/albertovpd/viu_tfm-deep_vision_classification/blob/main/src/TFM-serializing_data.ipynb . This is not optimal, and causes running out of RAM.

Then I started using tensorflow and its *image_from_dataset* method. It allows you to batch processing and also, use data augmentation for the batches, wich needs way less memory, but is quite tricky:
- by default, shuffle = True is set in *image_from_dataset*. That means that when using a classification report, labels in validaton dataset won't be associated with their right arrays, because it's shuffled. 
- Shuffle = False means that data is not shuffled at all, so while splitting, train_ds will have some of the classes, and val_ds will have other classes, it's like taking folders in order, without creating an heterogeneous sample for each one.

So, let's use some sklearn to create 2 heterogeneous folders: *train_folder* and *val_folder*.
- I create 2 folders because at first, the *image_from_dataset* method works just with 2 sets (not the typical train-test-validation). Than by the way can be modified later.
- They'll contain the same proportion than our dataset. For that purpose, we have to use **stratify** in sklarn.train_test_split.

In [35]:
# Google Drive stuff
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


- libraries

In [36]:
import os
import numpy as np
import shutil
import random

- paths

In [37]:
base_folder = "/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/"
input = base_folder+"input/House_Room_Dataset-5_rooms/" # for requesting directly pics
input_destination = base_folder+"input/dataset_2_folders/"

# splitting my data into just 2 folders

In [38]:
os.listdir(input)

['Kitchen', 'Livingroom', 'Bathroom', 'Bedroom', 'Dinning']

In [39]:
import os
import numpy as np
import shutil
import random
root_dir = base_folder+"input/House_Room_Dataset-5_rooms/" # for requesting directly pics
classes_dir = os.listdir(root_dir)

train_ratio = 0.6
val_ratio  = 0.1

for cls in classes_dir:
    os.makedirs(input_destination +'train_ds/' + cls, exist_ok=True)
    os.makedirs(input_destination +'test_ds/' + cls, exist_ok=True)
    os.makedirs(input_destination +'val_ds/' + cls, exist_ok=True)
    
    # for each class, let's counts its elements
    src = root_dir + cls
    allFileNames = os.listdir(src)

    # shuffle it and split into train/test/va
    np.random.shuffle(allFileNames)
    train_FileNames, test_FileNames, val_FileNames = np.split(np.array(allFileNames),[int(train_ratio * len(allFileNames)), int((1-val_ratio) * len(allFileNames))])
    
    # save their initial path
    train_FileNames = [src+'/'+ name  for name in train_FileNames.tolist()]
    test_FileNames  = [src+'/' + name for name in test_FileNames.tolist()]
    val_FileNames   = [src+'/' + name for name in val_FileNames.tolist()]
    print("\n *****************************",
          "\n Total images: ",cls, len(allFileNames),
          '\n Training: ', len(train_FileNames),
          '\n Testing: ', len(test_FileNames),
          '\n Validation: ', len(val_FileNames),
          '\n *****************************')
    
    # copy files from the initial path to the final folders
    for name in train_FileNames:
      shutil.copy(name, input_destination +'train_ds/' + cls)
    for name in test_FileNames:
      shutil.copy(name, input_destination +'test_ds/' + cls)
    for name in val_FileNames:
      shutil.copy(name, input_destination +'val_ds/' + cls)


 ***************************** 
 Total images:  Kitchen 965 
 Training:  579 
 Testing:  289 
 Validation:  97 
 *****************************

 ***************************** 
 Total images:  Livingroom 1273 
 Training:  763 
 Testing:  382 
 Validation:  128 
 *****************************

 ***************************** 
 Total images:  Bathroom 606 
 Training:  363 
 Testing:  182 
 Validation:  61 
 *****************************

 ***************************** 
 Total images:  Bedroom 1248 
 Training:  748 
 Testing:  375 
 Validation:  125 
 *****************************

 ***************************** 
 Total images:  Dinning 1158 
 Training:  694 
 Testing:  348 
 Validation:  116 
 *****************************


In [40]:
# checking everything was fine
paths = ['train_ds/', 'test_ds/','val_ds/']
for p in paths:
  for dir,subdir,files in os.walk(input_destination + p):
    print(dir,' ', p, str(len(files)))

/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/   train_ds/ 0
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/Kitchen   train_ds/ 579
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/Livingroom   train_ds/ 763
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/Bathroom   train_ds/ 363
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/Bedroom   train_ds/ 748
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/train_ds/Dinning   train_ds/ 694
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/test_ds/   test_ds/ 0
/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_2_folders/test_ds/Kitchen   test_ds/ 289
/content/drive/My Drive/2-Estudios/viu-mas